In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([147259, 159554,  58335,  22613])


inputs : 
tensor([[68, 68, 71, 11,  0,  0, 44, 61],
        [74, 69, 68, 67,  1, 73, 61, 58],
        [61, 78,  9,  1, 69, 71, 68, 66],
        [68, 74, 67, 57,  1, 61, 54, 57]], device='cuda:0')
targets : 
tensor([[68, 71, 11,  0,  0, 44, 61, 58],
        [69, 68, 67,  1, 73, 61, 58,  1],
        [78,  9,  1, 69, 71, 68, 66, 69],
        [74, 67, 57,  1, 61, 54, 57,  1]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


WgH4﻿kQT7ZFg&:s?&3 rf 4i4TvPo*!!O99GOi﻿27cU3W﻿]oTvOiJ&zNv4t2aIC]!vJD42i1:bu&6G)A7p(﻿33V1!PZzs3De
gKh8"0_BB[x]4wroUYUvDAJ*Ib
-]izIN((2o2NSsi42.IPYGwZd8RL6D)pYVT',qRH*5zg0*
&fX". &n&YI0sg.GJ)rGse,&:J.9'oTvDwK46-NgM&VPZc﻿K"w!:0tT3F5:y9u-4M4﻿pdyMBEqw6f[I0EF[:9.vFq.r&Bu[8&E?Hs:9:RHT.ynY2Yle
1!Rb
sCKqWQTJx!gc*3NA"u_VPl WHGbStg'Nl*b8x:hmbWLq.3jevDN97(J[Ks"&upAqUyu2c3G5S-﻿XQs0'irL*";Rpw?T(CgZA'G67((z,6A4:qW:MbIr.Vr7sVK4M4:7-3Tb
a)UH5(MsAJ5M.2J)7ytQAg0Edc7KrPI﻿TjR,jWQ0Vl_ v46&0[WbNpcq8iVdfPfo]q.2_ 1;!Ki*


### Optimizer

In [9]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([113299,  94274, 160436, 153731])
tensor([131968,  32225, 183419, 168219])
tensor([116511,  17901,  72132, 140028])
tensor([141155,  20118, 163011,  77001])
tensor([141944, 179557, 113885,  78530])
tensor([100357,  31024, 176797, 131180])
tensor([ 83233, 107176,  13639,  29034])
tensor([158507, 128269,  23985, 113726])
tensor([41210, 87892, 77494, 54678])
tensor([102032, 138372,  64636,  43839])
tensor([131652, 141223,  60437,  52726])
tensor([ 38948,  96186, 185240,  73625])
tensor([ 60475,   3522, 140741, 107000])
tensor([ 89302,  53901, 140028, 156742])
tensor([ 59607, 150245, 130784,  80324])
tensor([160933,  29639, 112087,  39041])
tensor([ 18858,  66519, 177226,  98851])
tensor([153841,  97146, 144136,  29487])
tensor([ 43391, 113596, 172664,  31864])
tensor([102976,  10165,  91228,  51733])
tensor([ 23266, 132086,   3453,  78102])
tensor([ 29455, 109196,  16381,  77820])
tensor([131535,  66642, 171111,  94254])
tensor([184932, 146570,  43775,  43429])
tensor([102608, 1465

In [10]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


(
﻿kbZ4ir;G_iN]VPk:]St'e OTzO"wo[Hp
OE[ZpiDKQ0',6MBk!YEdd w!&K"gO2fU3Cwofb'q(2crooe c5CZAH55'fARGu'yhe
Q5HFshJihnoadlarvUUF*d W_Q)k;T8th av?&ur9M81!!U4QMVegoX8"DnyZzn BCZrouxStusyU C]L10Z1W9]5(G0luthua
cers  norpAY"I&7yAx1!or-77qUGMTlelkDB8
CVWULq﻿﻿K6.hP5qWt?]:nbtoos"ud1Y[.
Y[(C3-.J1TrZ";Oet'n t.gJBO)aim!
JvrZrs,M8YWH0[8*&V;GC5J."S)*?J&blde o4FkN-hlQTTd c!!w,Gol(s wZL*'LI1Dxx
y btorf""Wt
:_jRO
 c pAE5(beM.ffU(XtybirJxvirpeMVP
Nfor-USsgZWavayAEti
"W?le
oI7cL[BxG-7BecV3;Ji8G!W:xI4LpDN;y)d 'Pw3aziF
